In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]= "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "1"; 
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [2]:
import numpy as np

In [3]:
from joblib import load
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense

Using TensorFlow backend.


In [4]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:1']

## Loading the dataset

In [5]:
X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
X_val = np.load('X_val.npy')

y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')
y_val = np.load('y_val.npy')

## Define the model

In [6]:
X_train.shape

(4346, 224, 224, 3)

In [7]:
y_train.shape

(4346, 97)

In [8]:
X_val.shape

(931, 224, 224, 3)

In [9]:
y_val.shape

(931, 97)

In [17]:
model = Sequential()

# First Conv Unit
model.add(
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3))
)
model.add(
    MaxPooling2D(pool_size=(2, 2))
)

# Second Conv Unit
model.add(
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu')
)
model.add(
    MaxPooling2D(pool_size=(2, 2))
)

# Add a dropout
model.add(Dropout(0.25))

# Third Conv Unit
model.add(
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu')
)
model.add(
    MaxPooling2D(pool_size=(2, 2))
)

# Fourth Conv Unit
model.add(
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu')
)
model.add(
    MaxPooling2D(pool_size=(2, 2))
)

# Add a dropout
model.add(Dropout(0.25))

# Flatten
model.add(Flatten())

# Dense
model.add(
    Dense(256, activation='relu')
)

# Add a dropout
model.add(Dropout(0.5))

# Output
model.add(
    Dense(97, activation='softmax')
)

In [18]:
model.summary();

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 109, 109, 32)      9248      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 54, 54, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 52, 52, 32)        9248      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 26, 26, 32)        0         
__________

## Compile the model

In [19]:
model.compile(loss='categorical_crossentropy', 
             optimizer='adam',
             metrics=['accuracy'])

## Training the model

In [15]:
history = model.fit(X_train, y_train, batch_size=128, epochs=10,
                   validation_data=(X_val, y_val))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 4346 samples, validate on 931 samples
Epoch 1/10
4346/4346 [==============================] - 9s 2ms/step - loss: 15.8640 - acc: 0.0145 - val_loss: 15.9103 - val_acc: 0.0129
Epoch 2/10
4346/4346 [==============================] - 5s 1ms/step - loss: 15.8807 - acc: 0.0147 - val_loss: 15.9103 - val_acc: 0.0129
Epoch 3/10
4346/4346 [==============================] - 4s 1ms/step - loss: 15.8890 - acc: 0.0140 - val_loss: 15.9450 - val_acc: 0.0107
Epoch 4/10
4346/4346 [==============================] - 5s 1ms/step - loss: 15.9364 - acc: 0.0113 - val_loss: 15.9450 - val_acc: 0.0107
Epoch 5/10
4346/4346 [==============================] - 5s 1ms/step - loss: 15.9463 - acc: 0.0104 - val_loss: 15.9450 - val_acc: 0.0107
Epoch 6/10
4346/4346 [==============================] - 5s 1ms/step - loss: 15.9789 - acc: 0.0085 - val_loss: 15.9450 - val_acc: 0.0107
Epoch 7/10
4346/4346 [=====================

We've got poor result because we haven't scaled the dataset. This is a live example of why we should always scale our dataset. Because the dataset was not scaled, the loss decreased for sometime and then it got stuck and hardly changed. We may also see an occational increase in the loss. Based on some observed situations, these are the following indications that the dataset requires scaling.

- Loss/Accuracy is not changing and over so many epochs, they are same
- Loss is even increasing or Accuracy is decreasing considering that the model is overfitting
- Model is taking a long time to train, even on GPUs

## Scaling and Retraining

In [20]:
X_train = X_train.astype('float32') / 255.
X_val = X_val.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

In [21]:
history = model.fit(X_train, y_train, batch_size=128, epochs=10,
                   validation_data=(X_val, y_val))

Train on 4346 samples, validate on 931 samples
Epoch 1/10
4346/4346 [==============================] - 6s 1ms/step - loss: 4.5209 - acc: 0.0412 - val_loss: 4.3979 - val_acc: 0.0623
Epoch 2/10
4346/4346 [==============================] - 5s 1ms/step - loss: 4.2989 - acc: 0.0838 - val_loss: 4.1282 - val_acc: 0.1300
Epoch 3/10
4346/4346 [==============================] - 5s 1ms/step - loss: 4.0185 - acc: 0.1337 - val_loss: 3.7999 - val_acc: 0.1772
Epoch 4/10
4346/4346 [==============================] - 4s 1ms/step - loss: 3.6606 - acc: 0.1963 - val_loss: 3.4950 - val_acc: 0.2406
Epoch 5/10
4346/4346 [==============================] - 5s 1ms/step - loss: 3.3450 - acc: 0.2478 - val_loss: 3.1396 - val_acc: 0.3093
Epoch 6/10
4346/4346 [==============================] - 5s 1ms/step - loss: 3.0724 - acc: 0.2941 - val_loss: 2.9685 - val_acc: 0.3652
Epoch 7/10
4346/4346 [==============================] - 5s 1ms/step - loss: 2.7419 - acc: 0.3543 - val_loss: 2.7557 - val_acc: 0.3921
Epoch 8/10
4346

But **42%** Validation accuracy is still not good enough. We need atleast a decent performace to use it in real life. But the problem is the data number of images are too small as compared to number of distinct class labels and with this small number of images, it is very difficult to achieve state of the performance **Except we use TRANSFER LEARNING**

## Test Performance

In [22]:
loss, accuracy = model.evaluate(X_test, y_test)

932/932 [==============================] - 1s 820us/step


In [23]:
loss

2.731557180953128

In [24]:
accuracy

0.3873390557939914

Even the performance on the test data set is around **39%**. You can also fine tune the model and see if it performs better or not but it's performance can hardly go beyond a certain limit becaue the number of images is so less.